# **Quantitative Value Strategy**

In volatile and uncertain market times, intrinsic value investing takes on less of a risk than momentum investing, which is a stronger strategy in a bull market. 



**"Value investing"** means investing in the stocks that are the cheapest relative to common measures of business value (like earnings or assets).

Value investing was opularized by investors like Warren Buffet, Seth Klarman, and Benjamin Graham.

For this project, we are going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

##### **Library Imports**

In [41]:
import numpy as np
import pandas as pd 
import xlsxwriter
import requests
from scipy import stats
import math

##### **Importing Our List of Stocks & API Token**

We will need to import our list of stocks and our API token before proceeding. We will make sure the .csv file is still in your working directory and import it with the following command:

**Documentation Page for Public API:** https://iexcloud.io/docs/api/

1. We need is a **base URL** for the API: a URL that will start every HTTP request. (You have to add which specific endpoints that you want to retrieve from the API. Most APIs only expose certain data through each endpoint, which makes things faster if you only need to retrieve certain data. Ex. If you have the Google Maps API, you might only want the name of the location, so you would send coordinates and get back its name. You would not get back other information like population.)

    a. **Sandbox mode of API:** randomized data, free
        1. Find **Testing Sandbox** on webpage

**Acquiring an API Token**

- Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

- API tokens (and other sensitive information) should be stored in a secrets.py file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

- Download your secrets.py file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [42]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secret import IEX_CLOUD_API_TOKEN

##### **Making Our First API Call**
- Making our version of our value screener
- We will start by building a simple value screener that ranks securities based on a single metric (price-to-earnings ratio)
    - price-earnings-ratio: price/yearly earnings forecast
        - price-earnings (peRatio) on IEXcloud
            - on Quote endpoint: look on JSON RESPONSE: shows 'latestPrice' and 'peRatio'
            - on Quote's HTTP REQUEST: GET /stock/{symbol}/quote/{field}, copy into notebook


In [43]:
symbol = 'aapl'

# api_url = https://sandbox.iexapis.com
# 1. Add stable to have stable version of API (as opposed to bleeding-edge version with \
    # features that  are not full tested)
# 2. take '{field}' off the end to parse entire JSON response instead of a single metric
# 3. Make it an f'
# 4. Add authentication token: interpolate with ?token = (iex cloud api token)
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

# Create API call using requests library
data = requests.get(api_url)

# print status code to show if HTTP request is succesful
print(data.status_code)


403


2:57:45
34:36